In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import bigquery

# Add the project root directory to Python path
sys.path.append(os.path.abspath(".."))

# Set up credentials explicitly
home = str(Path.home())
credentials_path = os.path.join(home, ".gcp", "credentials", "clv-dev-sa-key.json")
credentials = service_account.Credentials.from_service_account_file(
    credentials_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Set the credentials for use by client libraries
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

from src.pipeline.data_processor import DataProcessor
from src.config.__init__ import get_data_processing_config

# Get config and create processor
config = get_data_processing_config()
processor = DataProcessor(config=config['data_processing_config'])

# Run processing
processor.load_data()
processor.process_data()

# Get the processed data
processed_df = processor.get_processed_data()

# View first few rows
print("\nFirst few rows of the loaded data:")
print(processed_df.head())

# Display dataframe info
print("\nDataframe information:")
print(processed_df.info())

Successfully loaded 1,417,801 records from BigQuery
Starting data processing. Initial shape: (1417801, 23)
Cleaning data...
Records removed: 0

Quality Report:
Records processed: 1,229,407

Key Metrics:
frequency_mean: 1.90
recency_mean: 858.56
monetary_mean: 101.04

Quality Flags (% passing):
valid_frequency: 100.0%
valid_recency: 100.0%
valid_monetary: 100.0%
valid_dates: 100.0%
overall_valid: 100.0%

First few rows of the loaded data:
                                customer_id cohort_month  recency  frequency  \
10000  f4752eea-ccf9-3748-895f-f352a3351a2f   2024-11-01        1          1   
10001  59b596fd-7311-3ecf-9034-27cd21d9ba85   2024-11-01        1          1   
10002  e6230abe-a1c9-3ad6-87c3-d49b3152b287   2024-11-01        1          1   
10003  185fb521-68b0-3b3f-9f91-62def43bec94   2024-03-01      257          1   
10004  575b8df2-fd03-3fc7-8339-f55460ba7f71   2024-03-01      257          1   

       monetary  total_revenue  revenue_trend  avg_transaction_value  \
10000

In [2]:
# Print available columns
print("\nAvailable columns:")
print(processed_df.columns.tolist())


Available columns:
['customer_id', 'cohort_month', 'recency', 'frequency', 'monetary', 'total_revenue', 'revenue_trend', 'avg_transaction_value', 'first_purchase_date', 'last_purchase_date', 'customer_age_days', 'distinct_categories', 'distinct_brands', 'avg_interpurchase_days', 'has_online_purchases', 'has_store_purchases', 'total_discount_amount', 'avg_discount_amount', 'discount_rate', 'sms_active', 'email_active', 'is_loyalty_member', 'loyalty_points']


In [3]:
# Then proceed with the processing
processed_df.reset_index(drop=True, inplace=True)
processed_df['customer_index'] = processed_df.index

# Create lookup DataFrame with both columns
customer_lookup_df = processed_df[['customer_index', 'customer_id']].copy()

In [4]:
# Display the resulting DataFrame
print("\nCustomer lookup DataFrame:")
print(customer_lookup_df.head())

# Drop only the customer_index from processed_df
processed_df = processed_df.drop(columns=['customer_id', 'customer_index'])

# Display the updated processed_df structure
print("\nProcessed DataFrame info:")
print(processed_df.info())


Customer lookup DataFrame:
   customer_index                           customer_id
0               0  f4752eea-ccf9-3748-895f-f352a3351a2f
1               1  59b596fd-7311-3ecf-9034-27cd21d9ba85
2               2  e6230abe-a1c9-3ad6-87c3-d49b3152b287
3               3  185fb521-68b0-3b3f-9f91-62def43bec94
4               4  575b8df2-fd03-3fc7-8339-f55460ba7f71

Processed DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229407 entries, 0 to 1229406
Data columns (total 22 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   cohort_month            1229407 non-null  datetime64[ns]
 1   recency                 1229407 non-null  int64         
 2   frequency               1229407 non-null  Int64         
 3   monetary                1229407 non-null  float64       
 4   total_revenue           1229407 non-null  float64       
 5   revenue_trend           1229407 non-null  float64       
